# Default notebook

This default notebook is executed using Databricks Workflows as defined in resources/adflow_click_analaysis_and_modeling_job.yml.

In [ ]:
import mlflow
import mlflow.xgboost
import mlflow.pyfunc

from pyspark.sql import SparkSession, functions as F
from pyspark.sql.functions import col, hour, to_timestamp, lit, udf
from pyspark.sql.types import StringType

from xgboost import XGBClassifier
from sklearn.metrics import log_loss
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.calibration import CalibratedClassifierCV

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pandas.api.types import CategoricalDtype
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK, SparkTrials

import joblib
import logging
import sys
import json
import traceback
from typing import List, Dict, Any, Tuple
import yaml
import time
from datetime import datetime, timedelta


# Import libraries
%run reference
print(ENV_VARS)

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

In [2]:
def get_spark() -> SparkSession:
    try:
        from databricks.connect import DatabricksSession
        return DatabricksSession.builder.getOrCreate()
    except ImportError:
        return SparkSession.builder.getOrCreate()

In [ ]:
def get_current_user_email():
    try:
        return dbutils.notebook.entry_point.getDbutils().notebook().getContext().tags().apply('user')
    except Exception as e:
        logger.error(f"Failed to get user email: {str(e)}")
        return "unknown_user@fluentco.com"

In [ ]:
def execute_query(spark: SparkSession, run_id):
    # Create schemas if they don't exist
    spark.sql(f"CREATE SCHEMA IF NOT EXISTS centraldata_{ENV_VARS['ENV']}.datascience_model_deployment")
    
    # Common part of the query for both environments
    common_query = """
    SELECT
        timestamp,
        COALESCE(click, 0) AS click,
        COALESCE(profile_gender, gender) AS gender,
        campdata_campaignid AS campaign_id,
        trafsrc_sourceid AS traffic_source_id,
        trafsrc_trafficpartnerid AS traffic_partner_id,
        device_osname AS os_name,
        exact_age AS age,
        est_household_income AS household_income,
        campdata_flowimpressionposition AS position,
        date_part('HOUR', timestamp) AS hour
    FROM
        centraldata_prod.glue.adflow_mltesting_event_v2 a
    LEFT JOIN
        centraldata_prod.transunion.gold_email_details b
    ON
        a.profile_emailsha256 = b.sha256_email_address
    WHERE
        timestamp > date_add(DAY, -22, now()) 
    AND
        (campdata_flowimpressionposition <= 4 OR campdata_flowimpressionposition IS NULL)
    AND trafsrc_trafficpartnerid != '1ac29301-78ac-46c6-a79d-0632ad980e3a'
    """


    # Function to check if table exists
    def check_table_exists(env):
        table_exists = spark.sql(f"""
            SHOW TABLES IN centraldata_{env}.datascience_model_deployment
            LIKE 'adflow_click_training_data_version_2'
        """).count() > 0
        return table_exists


    # Function to execute and log query for each environment
    def execute_and_log_query(env, query):
        table_name = f"centraldata_{env}.datascience_model_deployment.adflow_click_training_data_version_2"

        # Check if the table exists, if not create it
        if not check_table_exists(env):
            spark.sql(f"CREATE TABLE {table_name} AS {common_query} LIMIT 0")
            print(f"Table '{table_name}' created.")
        else:
            print(f"Table '{table_name}' already exists.")

        # Count rows before insertion
        before_count_df = spark.sql(f"SELECT COUNT(*) as row_count FROM {table_name}")
        before_count = before_count_df.collect()[0]['row_count']

        mlflow.log_metric(f"rows_before_insert_{env}", before_count)

        # Insert the data into the table
        spark.sql(query)

        # Count rows after insertion
        after_count_df = spark.sql(f"SELECT COUNT(*) as row_count FROM {table_name}")
        after_count = after_count_df.collect()[0]['row_count']

        mlflow.log_metric(f"rows_after_insert_{env}", after_count)
        
        # Calculate the number of rows inserted
        rows_inserted = after_count - before_count

        # Log results to MLflow
        mlflow.log_metric(f"rows_inserted_{env}", rows_inserted)
        mlflow.log_param(f"data_query_{env}", query.strip())
        print(f"Query executed and {rows_inserted} rows inserted in {table_name}")

    # Query for the sandbox/dev schema as per env
    insert_query = f"""
    INSERT INTO centraldata_{ENV_VARS['ENV']}.datascience_model_deployment.adflow_click_training_data_version_2
    {common_query}
    AND timestamp > (SELECT COALESCE(MAX(timestamp), date_add(DAY, -22, now())) 
                     FROM centraldata_{ENV_VARS['ENV']}.datascience_model_deployment.adflow_click_training_data_version_2)
    """
    execute_and_log_query(ENV_VARS['ENV'], insert_query)


def main():
    spark = get_spark()

    # Get the current user's email to set the experiment name
    user_email = get_current_user_email()

    experiment_name = f"/Users/{user_email}/xgboost_{ENV_VARS['ENV']}"

    # # Set MLflow experiment
    mlflow.set_experiment(experiment_name)

    # Start MLflow run with the model name as the run name
    with mlflow.start_run(run_name=f"xgboost_{ENV_VARS['ENV']}_run") as run:
        run_id = run.info.run_id
        execute_query(spark, run_id)

        # Save the run_id for use in the second notebook
        dbutils.jobs.taskValues.set("run_id", run_id)
        print(f"Run ID saved: {run_id}")

    return run_id


if __name__ == "__main__":
    main()